# Ejercicio 2

In [ ]:
# Para ejercicio 2:
# Agarrar el archivo txt "Mecanica_del_juego.txt" separar cada determinada cantidad de caracteres. Ej: 200. Y en base a eso
# ir vectorizando

4

In [ ]:
from chonkie import SemanticChunker

# Inicializar el chunker con parámetros ajustados
chunker = SemanticChunker(
    embedding_model="minishlab/potion-base-8M",  # Modelo de embeddings por defecto
    mode="window",                              # Modo de comparación en ventana
    threshold=0.3,                              # Umbral de similitud para agrupar oraciones
    chunk_size=50,                              # Tamaño máximo del fragmento en tokens
    similarity_window=2,                        # Comparar con 2 oraciones anteriores
    min_sentences=1,                            # Mínimo de oraciones por fragmento
    min_characters_per_sentence=12,             # Mínimo de caracteres por oración
    min_chunk_size=2,                           # Mínimo de tokens por fragmento
    delim=['.', '!', '?', '\n'],                # Delimitadores para dividir oraciones
    return_type="chunks"                        # Retornar objetos de tipo chunk
)

# Cargar el archivo de texto
with open("Mecanica_del_juego.txt", "r", encoding="utf-8") as file:
    texto = file.read()

# Fragmentar el texto en trozos de 200 caracteres
chunks = chunker.chunk(texto)

# Mostrar los fragmentos generados
for i, chunk in enumerate(chunks):
    print(f"Fragmento {i + 1}:")
    print(f"Texto: {chunk.text}")
    print(f"Cantidad de tokens: {chunk.token_count}")
    print(f"Índices: {chunk.start_index} - {chunk.end_index}")
    print("---")


d:\Documentos del Administrador\Desktop\Facultad\Cuartas Materias\NLP\TP 1 Parte 2\NLP_Parte2\entorno_nlp_2\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
from sentence_transformers import SentenceTransformer

# Cargar el modelo de embeddings
modelo = SentenceTransformer('minishlab/potion-base-8M')

# Vectorizar cada fragmento
vectores = [modelo.encode(chunk.text) for chunk in chunks]

# Mostrar los vectores (solo mostramos los primeros 5 por ejemplo)
for i, vector in enumerate(vectores[:5]):
    print(f"Vector del fragmento {i + 1}: {vector[:10]}...")  # Mostramos los primeros 10 valores del vector


NameError: name 'chunks' is not defined

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from scipy.spatial.distance import euclidean, cityblock

def calcular_similitudes(vectores, query_vector):
    similitudes_coseno = cosine_similarity(vectores, [query_vector])
    similitudes_euclidiana = [euclidean(v, query_vector) for v in vectores]
    similitudes_manhattan = [cityblock(v, query_vector) for v in vectores]
    
    return similitudes_coseno.flatten(), similitudes_euclidiana, similitudes_manhattan


In [ ]:
# Frase de búsqueda que deseas comparar con los fragmentos
query = "explorador de naturaleza"

# Vectorizar la frase de búsqueda
query_vector = modelo.encode(query)

# Calcular las similitudes
similitudes_coseno, similitudes_euclidiana, similitudes_manhattan = calcular_similitudes(vectores, query_vector)

# Mostrar los resultados
for i, (cos, euclid, manhattan) in enumerate(zip(similitudes_coseno, similitudes_euclidiana, similitudes_manhattan)):
    print(f"Fragmento {i + 1}:")
    print(f"Similitud Coseno: {cos}")
    print(f"Distancia Euclidiana: {euclid}")
    print(f"Distancia Manhattan: {manhattan}")
    print("---")


In [ ]:
# Dudas: 
# Las justificaciones que deben estar en el informe, a que se refieren? es decir, que hay que justificar? 
# Debo cargar todo el repositorio o solo los archivos que voy a usar?